# Import libraries and connect to Google Drive 
Xuanqi HUANG's gg_drive

In [1]:
################################################# 
####  Check the file_path drive owner:xhuang ####     
#### -- CHECK THIS SHEET BEFORE RUNNING --   ####
#################################################
# Google Drive authentication 2 times verification
# Click the link and copy the google cloud SDK key
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
!pwd
!ls 

print("RAM info")
!free -m

import os 
import sys 
import pandas as pd 
import numpy as np 
import seaborn as sns
from matplotlib import pyplot as plt
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import subprocess
        import sys
        def install(package):
            subprocess.call([sys.executable, "-m", "pip", "install", package])
        install(package)
    finally:
        globals()[package] = importlib.import_module(package)
        
file_path = "/content/drive/My Drive/S5/F3B_415_G11/data"

# change path into this file
os.chdir(file_path) 

def del_var(var_name):
    del var_name
    gc.collect()
    return True

print("Data in the file is ")
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content
drive  sample_data
RAM info
              total        used        free      shared  buff/cache   available
Mem:          13022         430       11193           0        1398       12327
Swap:             0           0           0
Data in the file is 
absence.csv    categories.csv  ExportElevesUVSimple.csv  QCM3.csv
atelier.csv    draft.ipynb     fact_table_bi_exam.csv	 QCM.csv
catalogue.csv  eleve.csv       QCM2.csv			 question.csv


# Randomly Generate data

In [0]:
import random as rd

# generate QCM notes
df_cat = pd.read_csv('./catalogue.csv',names = ['question','sub'])
df_QCM = pd.read_csv('./QCM.csv')
df_QCM.drop(columns = ['nom'],inplace = True)
for i in range(2,4):
    questions = rd.sample(df_cat.question.values.tolist(),20)
#     print(len(questions))
    student_id = df_QCM.id_eleve.values.tolist()
    # colname = questions_q2.append('id_eleve')
    df_qcm = pd.DataFrame(np.random.randint(low=0, high=2, size=(len(student_id),20)),columns=questions)
    df_qcm.iloc[np.random.randint(low=0, high=len(student_id), size=5),:] = np.nan
    df_qcm.insert(0,'id_eleve',student_id)
    df_qcm.to_csv('QCM%d.csv'%i,index = False)


20
20


In [0]:
# generate absent page 
student_id = df_QCM.id_eleve.values.tolist()
colnames = ["atelier1","atelier2","atelier3"]
df_absence = pd.DataFrame(np.random.randint(low=1, high=101, size=(len(student_id),3)),columns=colnames)
df_absence[df_absence.iloc[:,:] < 90] = 0
df_absence[df_absence.iloc[:,:] >= 90] = 1
df_absence.sum(axis = 0)
df_absence.insert(0,'id_eleve',student_id)
df_absence.to_csv('absence.csv',index = False)

df_cat = pd.read_csv('./categories.csv')
df_cat['atelier'] = np.random.randint(low=1, high=4, size=df_cat.shape[0])
df_cat = df_cat.rename(columns = {"nom_sous-categorie":"sous_categorie",'nom_categorie':"categorie"})
df_cat[['sous_categorie','categorie','atelier']].to_csv('atelier.csv',index = False)

# Creat table of dimension-**questions**

In [0]:
def table_question(qcm_file,id_qcm):
  catalogue = pd.read_csv('catalogue.csv',header=None)
  catalogue.columns = ['code','name']
  categories = pd.read_csv('categories.csv')
  qcm = pd.read_csv(qcm_file)
  observations = [x for x in list(qcm.columns) if x not in ['nom', 'id_eleve']]

  cat = []
  sous_cat =[]
  for idx,i in enumerate(observations):

      name = catalogue.name[catalogue['code']== i].values.tolist()
      name = ''.join(name)
      categorie =categories['nom_categorie'][categories['code_sous-categorie']==name].values.tolist()
      categorie = ''.join(categorie)
      sous_categorie =categories['nom_sous-categorie'][categories['code_sous-categorie']==name].values.tolist()
      sous_categorie = ''.join(sous_categorie)
      cat.append(categorie)
      categorie = ''
      sous_cat.append(sous_categorie)
      sous_categorie=''
      name = ''

  data={'nom_question':observations,'sous_categorie': sous_cat,'categorie': cat,'id_qcm': id_qcm }
  question = pd.DataFrame(data)
  return question

num_qcm = 3
qcm_files = ['QCM.csv','QCM2.csv','QCM3.csv']
list_df = []
for i in range(num_qcm):
    question = table_question(qcm_files[i],i+1)
    list_df.append(question)

df_question = pd.concat(list_df,ignore_index = True)
df_question.insert(0,'id_question',[x for x in range(df_question.shape[0])])
df_question.to_csv('question.csv',index = False)

# Create table of fact

In [31]:
dict_df ={}
file_name = os.listdir()
for name in file_name:
    dict_df[name] = pd.read_csv('./'+name)
    
def bi_exam(qcm_file,id_qcm):     
    ## Table de fait:
    df_qcm = pd.read_csv('./'+qcm_file)
    df_question = pd.read_csv('./question.csv')
    df_student = pd.read_csv('./eleve.csv')

    colnames = ['id_eleve','id_question','note','absence','sous_categorie','categorie','id_groupe']
    dict_bi_exam = {}
    for col in colnames:
        dict_bi_exam[col] = []
    dict_bi_exam

    ## Questions
    questions_id = df_question.id_question.values.tolist()
    students_id = df_student['id_eleve'].values.tolist()

    for s in students_id:
        absence = 0
        if (df_qcm[df_qcm['id_eleve']==s].isna().sum(axis = 1).values[0]) >= 20: 
            absence = 1          
        for q in questions_id:
            if df_question['id_qcm'][(df_question['id_question'] == q)].values[0] == id_qcm:
                dict_bi_exam[colnames[0]].append(s)

                dict_bi_exam[colnames[2]].append(df_qcm[df_question['nom_question'][(df_question['id_question'] == q) & (df_question['id_qcm'] == id_qcm)]][df_qcm['id_eleve'] == s].values[0,0])
                dict_bi_exam[colnames[3]].append(absence)
                dict_bi_exam[colnames[1]].append(q)
                dict_bi_exam[colnames[4]].append(df_question["sous_categorie"][df_question['id_question'] == q].values[0])
                dict_bi_exam[colnames[5]].append(df_question["categorie"][df_question['id_question'] == q].values[0])
                dict_bi_exam[colnames[6]].append(df_student['groupe'][df_student['id_eleve']==s].values[0])

    # df_bi-exam = pd.DataFrame()
    a = pd.DataFrame(data = dict_bi_exam)
    a['note'][a.note.isna()] = -1
    return a
num_qcm = 3
qcm_files = ['QCM.csv','QCM2.csv','QCM3.csv']
list_df = []
for i in range(num_qcm):
    bi = bi_exam(qcm_files[i],i+1)
    list_df.append(bi)

df_bi = pd.concat(list_df,ignore_index = True)

df_bi.to_csv('fact_table_bi_exam.csv',index = False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Data Specification
In this part, we will verify all generated tables accroding to the source files.

1. verify **eleve.csv** with **ExportElevesUVSimple.csv**
2. verify **question.csv** 

    2.1 nom_question should be the same in each QCM. 
    
    2.2 sous_categorie and categorie should be the nom_sous-categorie and nom_categorie in the table categories.csv.


3.  Verify **fact_table_bi_exam.csv**

    3.1 **note** should be same as the score in each QCM file
    
    3.2 **absence** = 1 means the student is absent during this QCM, so the score of all questions in this QCM should be **Nan**  in the QCM file and -1 in this fact_table.






## Read tables

In [0]:
generated_path = ['eleve.csv','question.csv','fact_table_bi_exam.csv']
source_path = ['ExportElevesUVSimple.csv','catalogue.csv','categories.csv']
qcm_path = ['QCM.csv','QCM2.csv','QCM3.csv']

dict_df_new = {}
dict_df_old = {}
df_qcm_list = []

for name in generated_path:
    dict_df_new[name] = pd.read_csv(name)
    
for name in source_path:
    dict_df_old[name] = pd.read_csv(name)
    
for name in qcm_path:
    df_qcm_list.append(pd.read_csv(name))
    
dict_df_old['catalogue.csv'] = pd.read_csv('catalogue.csv',names = ['question','sub'])

## Verify eleve.csv

eleve.csv are built correctly

In [34]:
# Verification of eleve.csv
print(dict_df_new['eleve.csv'].head(5))
dict_df_old['ExportElevesUVSimple.csv'].head(5)

   id_eleve       nom       prenom  groupe niveau_initial_francais  \
0      6536  Blizzard    Banegrief       8                       0   
1      8389  Blizzard  Spelldragon       8                      b2   
2      6507   Bo'thal     Resthres       2                       0   
3      5961  Celeborn    Imorcaror       7                       0   
4     15671     Comet         Lord       8                       0   

  niveau_atteint_francais  
0                       0  
1                     B1+  
2                       0  
3                       0  
4                       0  


,ID_ELEVE,NOM_ELE,PRENOM_ELE,groupe_promo,num_groupe_etudiant,libelle,site,NOTE,TYPE_INSCR,CODE_UP,...,MOY_CC,MOY_CS,MOY,CREDIT,ECTS_GRADE,rang_domaine,code_domaine,NIVEAU_INIT_MAX_FRANCAIS,NIVEAU_ATTEINT_MAX_FRANCAIS,DATE_EDITION
0,6536,Blizzard,Banegrief,2,8,Jita,Brest,NaN,.,UVF3B415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26/01/2018
1,8389,Blizzard,Spelldragon,2,8,Jita,Brest,NaN,.,UVF3B415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b2,B1+,26/01/2018
2,6507,Bo'thal,Resthres,1,2,Évaluation,Brest,NaN,.,UVF3B415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26/01/2018
3,5961,Celeborn,Imorcaror,2,7,Évaluation,Brest,NaN,.,UVF3B415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26/01/2018
4,15671,Comet,Lord,2,8,Jita,Brest,NaN,.,UVF3B415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26/01/2018


## Verify question.csv

question.csv is bult correctly

In [58]:
# verification of question.csv
def find_question_info(line, dict_df_old = dict_df_old):
    df = dict_df_old['catalogue.csv']
    df2 = dict_df_old['categories.csv']
    if df2['nom_sous-categorie'][df2['code_sous-categorie'] == df['sub'][df.question == line.nom_question].values[0]].values[0] != line.sous_categorie:
        print('wrong line in question.csv :\n',line)
        print('should be : ',df['sub'][df.question == line.nom_question].values[0])
    return True 
print('If has wrong value, it will indicate the wrong value and right value')
a = dict_df_new['question.csv'].apply(lambda line: find_question_info(line), axis = 1)
print('verification finished')
# dict_df_new['question.csv']

If has wrong value, it will indicate the wrong value and right value
verification finished


question.csv are built correctly

## Verify fact_table_bi_exam.csv